# Offshore Weather Route Analysis

This notebook demonstrates comprehensive weather analysis for offshore sailing routes using NOAA model data (GFS, GEFS, WW3).

## Scenario
Analyze departure windows for vessels sailing from Hampton Roads to Bermuda and/or Antigua, evaluating:
- Cut-off upper-level lows and their evolution
- Wind and wave conditions along the route
- Gulf Stream crossing recommendations
- Risk assessment for different vessel speeds
- Departure window recommendations

## Setup

In [ ]:
# Import libraries
import sys
sys.path.insert(0, '../src')

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import wx_anal modules
from wx_anal.config import Config
from wx_anal.downloader import WeatherDownloader
from wx_anal.analyzer import WeatherAnalyzer
from wx_anal.routes import Route, Vessel, VesselSpeed, GulfStream

# Setup
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

print("✓ Libraries loaded successfully")

## Configuration

Set up analysis parameters for the departure window analysis.

In [ ]:
# Analysis parameters
DEPARTURE_DATE = datetime(2025, 10, 31, 18, 0)  # Friday afternoon departure
ALTERNATE_DATE = datetime(2025, 11, 5, 12, 0)   # Wednesday midday departure

ROUTE_NAME = "hampton-bermuda"
FORECAST_DAYS = 10

# Create configuration
config = Config(data_dir="../data")

print(f"Scenario: Departure window analysis")
print(f"Primary departure: {DEPARTURE_DATE.strftime('%A, %B %d, %Y at %H:%M UTC')}")
print(f"Alternate departure: {ALTERNATE_DATE.strftime('%A, %B %d, %Y at %H:%M UTC')}")
print(f"Route: {ROUTE_NAME}")
print(f"Forecast period: {FORECAST_DAYS} days")

## Define Vessels and Routes

Create vessel profiles for different speed categories (as discussed in the forecast briefing).

In [ ]:
# Create vessel profiles
slow_boat = Vessel.slow_boat("Slow Cruiser")
typical_boat = Vessel.typical_boat("Typical Cruiser")
fast_boat = Vessel.fast_boat("Fast Cruiser")

vessels = [slow_boat, typical_boat, fast_boat]

# Display vessel characteristics
vessel_df = pd.DataFrame([
    {
        "Category": v.speed_category.value,
        "Avg Speed (kt)": v.avg_speed_knots,
        "NM/Day": v.nm_per_day,
        "Max Wind (kt)": v.max_wind_tolerance,
        "Max Waves (m)": v.max_wave_tolerance,
    }
    for v in vessels
])

print("Vessel Performance Profiles:")
print(vessel_df.to_string(index=False))

# Create routes
routes = {}
for vessel in vessels:
    routes[vessel.name] = Route(ROUTE_NAME, vessel=vessel)

typical_route = routes["Typical Cruiser"]
distance = typical_route._calculate_total_distance()
print(f"\nRoute distance: {distance:.0f} nautical miles")

## Download Weather Data

Fetch GFS, GEFS, and WW3 data from NOAA NOMADS.

**Note:** This cell attempts to download real data from NOAA. If the connection fails, the analysis will continue with limited data.

In [ ]:
downloader = WeatherDownloader(config)

print("Attempting to download weather model data from NOAA NOMADS...")
print("This may take several minutes.\n")

try:
    # Download data
    weather_data = downloader.download_offshore_route_data(
        route_name=ROUTE_NAME,
        run_date=DEPARTURE_DATE,
        forecast_days=FORECAST_DAYS,
    )
    
    print("✓ Data download complete")
    print(f"  GFS: {'✓' if weather_data.get('gfs') else '✗'}")
    print(f"  GEFS: {'✓' if weather_data.get('gefs') else '✗'}")
    print(f"  WW3: {'✓' if weather_data.get('ww3') else '✗'}")
    
except Exception as e:
    print(f"⚠️  Could not download data: {e}")
    print("\nNote: NOAA NOMADS OPeNDAP access requires:")
    print("  - Active internet connection")
    print("  - Recent model run availability")
    print("  - Proper netCDF4/cfgrib installation")
    print("\nContinuing with demonstration mode...")
    weather_data = {"gfs": None, "gefs": None, "ww3": None}

## Summary

This notebook provides a framework for offshore weather route analysis. Key features:

1. ✓ Vessel speed categories (slow/typical/fast boats)
2. ✓ Pre-defined routes (Hampton-Bermuda, Hampton-Antigua, etc.)
3. ✓ Weather data downloading framework (GFS, GEFS, WW3)
4. ✓ Feature detection (cut-off lows, jet streams)
5. ✓ Route hazard analysis (winds, waves)
6. ✓ Risk assessment and scoring
7. ✓ Departure window recommendations

### CLI Usage

```bash
# Run from command line
wx-anal --route hampton-bermuda --start 2025-10-31 --speed typical
```

### Next Steps

To complete the analysis shown in subsequent cells:
1. Ensure you have internet connectivity for NOAA data
2. Install all required dependencies: `pip install -e .[notebook]`
3. Run cells sequentially to perform the full analysis